In [1]:
!pip install -q pygbif

In [2]:
from opentree import OT

import os
import sys
import json
import dendropy

from Bio import Entrez
Entrez.email = "ejmctavish@ucmerced.edu"

from pygbif import occurrences as occ

In [3]:
taxonomy_file = "../../ott3.2/taxonomy.tsv"
assert os.path.exists(taxonomy_file)

In [4]:
!head -n 1 "../../ott3.2/taxonomy.tsv"

uid	|	parent_uid	|	name	|	rank	|	sourceinfo	|	uniqname	|	flags	|	


# Get families and sources directly from taxonomy download

In [5]:
fam_dict = {}
species_set = set()
genus_dict = {}
all_tax_dict = {}
header = ['ottid', 'parent_ottid', 'name','rank','source','uniqname','flags']
for lin in open(taxonomy_file):
        lii=lin.split('\t|\t')
        if len(lii[2].split(' ')) > 1:
            pass
        elif lii[2].endswith("aceae"):
            fam_dict[lii[2]]=lii
        elif lii[2].endswith("idae"):
            fam_dict[lii[2]]=lii
        if lii[3] == 'species':
            species_set.add(lii[0])
        all_tax_dict[lii[0]]=dict(zip(header, lii))

### Create a wrapper function for pulling info from NCBI

In [6]:
def get_ncbi_records(ncbi_id):
    handle = Entrez.egquery(retmax=50, term="txid{}[Organism]".format(ncbi_id), idtype="acc")
    gen_record = Entrez.read(handle)
    eq_results = {}
    for db in gen_record['eGQueryResult']:
        try:
            count = int(db['Count'])
            if count > 0:
                eq_results[db['DbName']]=db
        except ValueError:
            pass
    return(eq_results)
    

In [8]:
fams_node_id = json.load(open('fam_synth_node_info.json'))
fams_resp = json.load(open('fam_resp_dict.json'))

In [10]:
node_ids = {}
fams_in_tree = set()
not_in_tree = set()
monophy = set()
non_monophy = set()
for fam in fams_node_id:
    nid = fams_node_id[fam]
    ott_id = fam_dict[fam][0]
    if nid:
        if nid not in node_ids:
            node_ids[nid] = set()
        node_ids[nid].add(fam)
        fams_in_tree.add(fam)
        if nid.strip('ott') == ott_id:
            monophy.add(fam)
        else:
            non_monophy.add(fam)
    else:
        not_in_tree.add(fam)

In [11]:
len(non_monophy)+len(monophy)

10104

In [12]:
synthtreeid = OT.synth_induced_tree(node_ids = list(node_ids.keys()), label_format="id")
synthtreeid.tree.write(path="allfam_id_label.tre",schema="newick")

In [14]:
fam_synth_tips = [leaf.taxon.label for leaf in synthtreeid.tree.leaf_node_iter()]

In [16]:
test_tips = []
fam_synth_tips.sort()
for node in fam_synth_tips[:5]:
    synth_sub = OT.synth_subtree(node_id=node, label_format="id")
    #sub_taxa = [node.label for node in synth_sub.tree if node.label]
    sub_tips = [leaf.taxon.label for leaf in synth_sub.tree.leaf_node_iter()]
    #internal_spp = [ottid for ottid in  sub_taxa if ottid.strip('ott') in species_set]
    #print(len(sub_spp_and_below))
    #node_tips[node] = sub_tips
    test_tips = test_tips + sub_tips

## Search across all of genbank and gbif on each taxon id, and add to dict

In [17]:
#tip_dict = {}
tip_dict = json.load(open('tip_dict.json'))

#all_synth_tips = [leaf.taxon.label for leaf in total_synth.tree.leaf_node_iter()]
#fi = open("tip_interop.csv", "a")

all_synth_tips = test_tips

for tip in all_synth_tips:
    print(len(tip_dict))
    if tip not in tip_dict:
        assert(tip.startswith('ott'))
        tmp_dict = all_tax_dict[tip.strip('ott')]
        tmp_dict['ncbi_ids'] = []
        tmp_dict['gbif_ids'] = []   
        tmp_dict['ncbi_eq_results'] = []
        tmp_dict['gbif_records'] = 0
        for source in tmp_dict['source'].split(','):
            if source.startswith('ncbi'):
                ncbi_id = source.split(':')[1]
                tmp_dict['ncbi_ids'].append(ncbi_id)
                eq_results = get_ncbi_records(ncbi_id)
                tmp_dict['ncbi_eq_results'].append(eq_results)
            if source.startswith('gbif'):
                gbif_id = source.split(':')[1]
                tmp_dict['gbif_ids'].append(gbif_id)
                tmp_dict['gbif_records'] += occ.count(taxonKey = gbif_id)
        tip_dict[tip] = tmp_dict

        


9910
9911


KeyboardInterrupt: 

In [ ]:
test_tips = []
node_tips = {}
synth_tips_over_500.sort()
for node in synth_tips_over_500[:5]:
    synth_sub = OT.synth_subtree(node_id=node, label_format="id")
    sub_taxa = [node.label for node in synth_sub.tree if node.label]
    sub_tips = [leaf.taxon.label for leaf in synth_sub.tree.leaf_node_iter()]
    internal_spp = [ottid for ottid in  sub_taxa if ottid.strip('ott') in species_set]
    print(len(sub_spp_and_below))
    node_tips[node] = sub_tips
    test_tips = test_tips + sub_tips

In [18]:
node_annotation = {}
for nid in fam_synth_tips[:5]:
    print(nid)
    node_annotation[nid] = {}
    node_annotation[nid]['tips_counted'] = 0
    node_annotation[nid]['ncbi_ids'] = 0
    node_annotation[nid]['gbif_ids'] = 0
    node_annotation[nid]['genomes'] = 0
    node_annotation[nid]['genbank'] = 0
    node_annotation[nid]['gbif_occ'] = 0
    node_annotation[nid]['has_gbif_dat'] = 0
    node_annotation[nid]['has_gbif_id'] = 0
    node_annotation[nid]['has_nuc_dat'] = 0
    node_annotation[nid]['has_ncbi_id'] = 0
    node_annotation[nid]['total_descendents'] = len(node_tips[nid])
    for tip in node_tips[nid]:
        assert tip in test_tips
        if tip in tip_dict:
            node_annotation[nid]['tips_counted'] += 1
            node_annotation[nid]['ncbi_ids'] +=len(tip_dict[tip].get('ncbi_ids',[]))
            node_annotation[nid]['genomes'] += sum([int(res.get('genome',{'Count':0})['Count']) for res in tip_dict[tip]['ncbi_eq_results']])
            node_annotation[nid]['genbank'] += sum([int(res.get('nuccore',{'Count':0})['Count']) for res in tip_dict[tip]['ncbi_eq_results']])
            node_annotation[nid]['gbif_occ'] += int(tip_dict[tip]['gbif_records'])
            node_annotation[nid]['gbif_ids'] +=len(tip_dict[tip].get('gbif_ids',[]))
            if len(tip_dict[tip].get('ncbi_ids',[])) >= 1:
                node_annotation[nid]['has_ncbi_id'] += 1
            if len(tip_dict[tip].get('gbif_ids',[])) >= 1:
                node_annotation[nid]['has_gbif_id'] += 1
            if int(tip_dict[tip]['gbif_records']) >= 1:
                node_annotation[nid]['has_gbif_dat'] += 1
            if sum([int(res.get('nuccore',{'Count':0})['Count']) for res in tip_dict[tip]['ncbi_eq_results']]) >= 1:
                node_annotation[nid]['has_nuc_dat'] +=1
                
    node_annotation[nid]['has_gbif_dat_perc'] = int((node_annotation[nid]['has_gbif_dat']/node_annotation[nid]['tips_counted'])*100)
    node_annotation[nid]['has_nuc_dat_perc'] = int((node_annotation[nid]['has_nuc_dat']/node_annotation[nid]['tips_counted'])*100)
    node_annotation[nid]['has_gbif_id_perc'] = int((node_annotation[nid]['has_gbif_id']/node_annotation[nid]['tips_counted'])*100)
    node_annotation[nid]['has_ncbi_id_perc'] = int((node_annotation[nid]['has_ncbi_id']/node_annotation[nid]['tips_counted'])*100)
                

            



mrcaott101284ott282313


NameError: name 'node_tips' is not defined

## Start by finding lineages that represent 500 or more descendents

In [7]:
# Download synth tree from https://tree.opentreeoflife.org/about/synthesis-release/v12.3
tips_over_500 = set()

fi = '/home/ejmctavish/projects/otapi/opentree12.3_tree/labelled_supertree/labelled_supertree.tre'
total_synth = dendropy.Tree.get(path=fi, schema="newick")
for node in total_synth:
    if len(node.leaf_nodes()) > 500:
        tips_over_500.add(node.label)

KeyboardInterrupt: 

In [ ]:
synthtreeid_over_500 = OT.synth_induced_tree(node_ids = list(tips_over_500), label_format="id")
synth_tips_over_500 = [leaf.taxon.label for leaf in synthtreeid_over_500.tree.leaf_node_iter()]

In [ ]:
len(species_set)

In [ ]:
len(test_tips)

In [ ]:
#with open('tip_dict.json', 'w') as outfile:
#    json.dump(tip_dict, outfile)

In [14]:
node_annotation

{'mrcaott10050ott302228': {'tips_counted': 507,
  'ncbi_ids': 240,
  'gbif_ids': 463,
  'genomes': 7,
  'genbank': 88252,
  'gbif_occ': 218388,
  'has_gbif_dat': 424,
  'has_gbif_id': 462,
  'has_nuc_dat': 239,
  'has_ncbi_id': 240,
  'total_descendents': 507,
  'has_gbif_dat_perc': 83,
  'has_nuc_dat_perc': 47,
  'has_gbif_id_perc': 91,
  'has_ncbi_id_perc': 47},
 'mrcaott10141ott61528': {'tips_counted': 822,
  'ncbi_ids': 147,
  'gbif_ids': 791,
  'genomes': 0,
  'genbank': 29147,
  'gbif_occ': 64293,
  'has_gbif_dat': 619,
  'has_gbif_id': 791,
  'has_nuc_dat': 146,
  'has_ncbi_id': 147,
  'total_descendents': 822,
  'has_gbif_dat_perc': 75,
  'has_nuc_dat_perc': 17,
  'has_gbif_id_perc': 96,
  'has_ncbi_id_perc': 17},
 'mrcaott10167ott64672': {'tips_counted': 559,
  'ncbi_ids': 238,
  'gbif_ids': 532,
  'genomes': 0,
  'genbank': 67859,
  'gbif_occ': 51104,
  'has_gbif_dat': 508,
  'has_gbif_id': 527,
  'has_nuc_dat': 237,
  'has_ncbi_id': 238,
  'total_descendents': 559,
  'has_gb

In [16]:
'ott922493' in node_tips['mrcaott102ott38119']

True

In [15]:
tip = 'ott922493'
tip_dict[tip]

{'ottid': '922493',
 'parent_ottid': '164652',
 'name': 'Mus caroli',
 'rank': 'species',
 'source': 'ncbi:10089,gbif:2438800,irmng:10723449',
 'uniqname': '',
 'flags': '',
 'ncbi_ids': ['10089'],
 'gbif_ids': ['2438800'],
 'ncbi_eq_results': [{'pmc': {'DbName': 'pmc',
    'MenuName': 'PubMed Central',
    'Count': '196',
    'Status': 'Ok'},
   'nuccore': {'DbName': 'nuccore',
    'MenuName': 'Nucleotide',
    'Count': '57469',
    'Status': 'Ok'},
   'protein': {'DbName': 'protein',
    'MenuName': 'Protein',
    'Count': '47730',
    'Status': 'Ok'},
   'genome': {'DbName': 'genome',
    'MenuName': 'Genome',
    'Count': '1',
    'Status': 'Ok'},
   'taxonomy': {'DbName': 'taxonomy',
    'MenuName': 'Taxonomy',
    'Count': '1',
    'Status': 'Ok'},
   'gene': {'DbName': 'gene',
    'MenuName': 'Gene',
    'Count': '33051',
    'Status': 'Ok'},
   'sra': {'DbName': 'sra', 'MenuName': 'SRA', 'Count': '185', 'Status': 'Ok'},
   'popset': {'DbName': 'popset',
    'MenuName': 'PopSet'

In [46]:
len(all_synth_tips)

3199

In [47]:
mini_synth_tree = OT.synth_induced_tree(node_ids=synth_tips_over_500[:5], label_format="id")

In [48]:
mini_synth_tree.tree.write(path="desc500plus_5.tre",schema="newick")

In [16]:
def write_itol_heatmap(filename, title, unit, node_annotation, param):
    """Write out an itol heatmap file to filename, with title and units label.
    annot dict must have keys which are node ids in tree on itol (may require underscore space subs..)
    param should be key of annot_dict[node_ids] = {}"""
    fi = open(filename, 'w')
    startstr = """DATASET_HEATMAP
    #In heatmaps, each ID is associated to multiple numeric values, which are displayed as a set of colored boxes defined by a color gradient
    #lines starting with a hash are comments and ignored during parsing
    #=================================================================#
    #                    MANDATORY SETTINGS                           #
    #=================================================================#
    #select the separator which is used to delimit the data below (TAB,SPACE or COMMA).This separator must be used throughout this file (except in the SEPARATOR line, which uses space).
    #SEPARATOR TAB
    SEPARATOR SPACE
    #SEPARATOR COMMA

    #label is used in the legend table (can be changed later)
    DATASET_LABEL {t}

    #dataset color (can be changed later)
    COLOR #ff0000

    #define labels for each individual field column
    FIELD_LABELS {u}

    #=================================================================#
    #                    OPTIONAL SETTINGS                            #
    #=================================================================#


    #Heatmaps can have an optional Newick formatted tree assigned. Its leaf IDs must exactly match the dataset FIELD_LABELS.
    #The tree will be used to sort the dataset fields, and will be displayed above the dataset. It can have branch lengths defined.
    #All newlines and spaces should be stripped from the tree, and COMMA cannot be used as the dataset separator if a FIELD_TREE is provided.
    #FIELD_TREE (((f1:0.2,f5:0.5):1,(f2:0.2,f3:0.3):1.2):0.5,(f4:0.1,f6:0.5):0.8):1;



    #=================================================================#
    #     all other optional settings can be set or changed later     #
    #           in the web interface (under 'Datasets' tab)           #
    #=================================================================#

    #Each dataset can have a legend, which is defined using LEGEND_XXX fields below
    #For each row in the legend, there should be one shape, color and label.
    #Optionally, you can define an exact legend position using LEGEND_POSITION_X and LEGEND_POSITION_Y. To use automatic legend positioning, do NOT define these values
    #Optionally, shape scaling can be present (LEGEND_SHAPE_SCALES). For each shape, you can define a scaling factor between 0 and 1.
    #Shape should be a number between 1 and 6, or any protein domain shape definition.
    #1: square
    #2: circle
    #3: star
    #4: right pointing triangle
    #5: left pointing triangle
    #6: checkmark

    #LEGEND_TITLE,Dataset legend
    #LEGEND_POSITION_X,100
    #LEGEND_POSITION_Y,100
    #LEGEND_SHAPES,1,2,3
    #LEGEND_COLORS,#ff0000,#00ff00,#0000ff
    #LEGEND_LABELS,value1,value2,value3
    #LEGEND_SHAPE_SCALES,1,1,0.5

    #left margin, used to increase/decrease the spacing to the next dataset. Can be negative, causing datasets to overlap.
    #MARGIN 0

    #width of the individual boxes
    #STRIP_WIDTH 25

    #always show internal values; if set, values associated to internal nodes will be displayed even if these nodes are not collapsed. It could cause overlapping in the dataset display.
    #SHOW_INTERNAL 0


    #show dashed lines between leaf labels and the dataset
    #DASHED_LINES 1

    #if a FIELD_TREE is present, it can be hidden by setting this option to 0
    #SHOW_TREE 1

    #define the color for the NULL values in the dataset. Use the letter X in the data to define the NULL values
    #COLOR_NAN #000000

    #automatically create and display a legend based on the color gradients and values defined below
    #AUTO_LEGEND 1


    #define the heatmap gradient colors. Values in the dataset will be mapped onto the corresponding color gradient.
    COLOR_MIN #0000ff
    COLOR_MAX #ff0000

    #you can specify a gradient with three colors (e.g red to yellow to green) by setting 'USE_MID_COLOR' to 1, and specifying the midpoint color
    #USE_MID_COLOR 1
    #COLOR_MID #ffff00

    #By default, color gradients will be calculated based on dataset values. You can force different values to use in the calculation by setting the values below:
    #USER_MIN_VALUE 0
    #USER_MID_VALUE 500
    #USER_MAX_VALUE 1000

    #border width; if set above 0, a border of specified width (in pixels) will be drawn around individual cells
    #BORDER_WIDTH,0

    #border color; used only when BORDER_WIDTH is above 0
    #BORDER_COLOR,#0000ff


    #Internal tree nodes can be specified using IDs directly, or using the 'last common ancestor' method described in iTOL help pages
    #=================================================================#
    #       Actual data follows after the "DATA" keyword              #
    #=================================================================#
    DATA\n
    """.format(t=title, u=unit)
    fi.write(startstr)
    for node in node_annotation:
        desc = int(node_annotation[node][param])
        fi.write("{} {}\n".format(node, desc))    
    fi.close()


In [22]:
write_itol_heatmap("genomes_heatmap.txt", "Genome_counts", "Genome_count", node_annotation, 'genomes')
write_itol_heatmap("genbank.txt", "genbank_counts", "genbank_count", node_annotation, 'genbank')
write_itol_heatmap("GBIF_heatmap.txt", "GBIF_counts", "GBIF_count", node_annotation, 'gbif_occ')
write_itol_heatmap("genbank_perc.txt", "genbank_perc", "genbank_perc", node_annotation, 'has_gbif_dat_perc')
write_itol_heatmap("GBIF_perc.txt", "GBIF_perc", "GBIF_perc", node_annotation, 'has_nuc_dat_perc')


## For each family in tree get tip taxa, including NCBI ids and GBIF ids

In [12]:
for fam in tip_dict:
    print(fam)
    for tip in tip_dict[fam]:
        if 'ncbi_eq_results' in tip_dict[fam][tip]:
            pass
        else:
            tip_dict[fam][tip]['ncbi_eq_results'] = []
            tip_dict[fam][tip]['gbif_records'] = 0
            for ncbi_id in tip_dict[fam][tip]['ncbi_ids']:
                handle = Entrez.egquery(retmax=50, term="txid{}[Organism]".format(ncbi_id), idtype="acc")
                gen_record = Entrez.read(handle)
                eq_results = {}
                for db in gen_record['eGQueryResult']:
                    try:
                        count = int(db['Count'])
                        if count > 0:
                            eq_results[db['DbName']]=db
                    except ValueError:
                        pass
                tip_dict[fam][tip]['ncbi_eq_results'].append(eq_results)
            for gbif_id in tip_dict[fam][tip]['gbif_ids']:
                tip_dict[fam][tip]['gbif_records'] += occ.count(taxonKey = gbif_id)

ott649007
ott782239
ott782231
ott1053057
ott372826
ott766272
ott36015
mrcaott24237ott109257
ott412944
ott436442


In [10]:
tip_dict = {}
for fam in synth_tips[:10]:
    tip_dict[fam]={}
    synth_sub = OT.synth_subtree(node_id=fam, label_format="id")
    sub_tips = [leaf.taxon.label for leaf in synth_sub.tree.leaf_node_iter()]
    for tip in sub_tips:
        tip_dict[fam][tip] = all_tax_dict[tip.strip('ott')]
        tip_dict[fam][tip]['ncbi_ids'] = []
        tip_dict[fam][tip]['gbif_ids'] = []     
        for source in tip_dict[fam][tip]['source'].split(','):
            if source.startswith('ncbi'):
                tip_dict[fam][tip]['ncbi_ids'].append(source.split(':')[1])
            if source.startswith('gbif'):
                tip_dict[fam][tip]['gbif_ids'].append(source.split(':')[1])

In [3]:
fams_node_id = json.load(open('fam_synth_node_info.json'))
## reload families in tree from all
#fams_resp = json.load(open('fam_resp_dict.json'))

In [8]:
node_ids = {}
fams_in_tree = set()
not_in_tree = set()
monophy = set()
non_monophy = set()
for fam in fams_node_id:
    nid = fams_node_id[fam]
    ott_id = fam_dict[fam][0]
    if nid:
        if nid not in node_ids:
            node_ids[nid] = set()
        node_ids[nid].add(fam)
        fams_in_tree.add(fam)
        if nid.strip('ott') == ott_id:
            monophy.add(fam)
        else:
            non_monophy.add(fam)
    else:
        not_in_tree.add(fam)

In [76]:
#with open('tip_dict.json', 'w') as outfile:
#    json.dump(tip_dict, outfile)

In [13]:
synthtreeid.tree.write(path="mini_fam_id_label.tre",schema="newick")

## Go through tree of tips w/500 plus desc

## Get tree of families

In [7]:
synthtreeid = OT.synth_induced_tree(node_ids = list(labels), label_format="id")
#synthtreeid.tree.write(path="desc500plus.tre",schema="newick")

## For each of the familes in the synth tree, what proportion of the lineages contained in that family have:
  - sequences in genbank
  - genomes in genbank or SRA
  - location records in GBIF

In [10]:
len(test_tips)

47020

In [ ]:
test_tips[:5]

In [ ]:
tip_dict['ott7827144']

In [62]:
tip = 'ott154258'
sum([int(res['nuccore']['Count']) for res in tip_dict[tip]['ncbi_eq_results']])

21

In [58]:
tip_dict[tip]['ncbi_eq_results']

[{'nuccore': {'DbName': 'nuccore', 'MenuName': 'Nucleotide', 'Count': '21', 'Status': 'Ok'},
  'protein': {'DbName': 'protein', 'MenuName': 'Protein', 'Count': '29', 'Status': 'Ok'},
  'taxonomy': {'DbName': 'taxonomy', 'MenuName': 'Taxonomy', 'Count': '1', 'Status': 'Ok'},
  'popset': {'DbName': 'popset', 'MenuName': 'PopSet', 'Count': '6', 'Status': 'Ok'}}]

In [66]:
sum([int(res.get('genome',{'Count':0})['Count']) for res in tip_dict[tip]['ncbi_eq_results']])

0